# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize

C:\Users\idoli\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
se = SearchEngine()

In [3]:
se.set_searcher(fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=300) #MAP=0.264891

In [4]:
topics = load_topics("Data/queriesROBUST.txt")

Extract Train Set Results

In [5]:
HARD_QUERIES =[309, 308, 338, 344, 348, 320, 328, 334, 303, 339] # From EDA

In [ ]:
all_hits = {}
hard_hits = {}
for i, (qid, topic) in enumerate(topics.items()):
    results = se.get_top_k(topic, k=1000, clean=True)
    all_hits[f"{qid}_{topic}"] = results
    if int(qid) in HARD_QUERIES:
        hard_hits[f"{qid}_{topic}"] = results
    if i==49:
        print(qid)
        break

import pickle
with open("pkls/top1000_rm3_train.pkl", "wb") as f:
    pickle.dump(all_hits, f)
with open("pkls/top1000_rm3_train_hard.pkl", "wb") as f:
    pickle.dump(hard_hits, f)

In [ ]:
res = se.retrieve_rerank("international organized crime", k=1000, m=50)

In [4]:
res

[('FBIS4-7811', np.float32(0.26468754)),
 ('FBIS4-46734', np.float32(0.13777173)),
 ('FBIS3-24145', np.float32(-1.8336121)),
 ('FBIS4-40260', np.float32(-3.4642494)),
 ('FBIS4-39330', np.float32(-5.274441)),
 ('FBIS3-23986', 6.779200077056885),
 ('FBIS3-37418', 6.731599807739258),
 ('FBIS4-41991', 6.730400085449219),
 ('FBIS4-16951', 6.632299900054932),
 ('FBIS4-22471', 6.598800182342529)]

In [4]:
# from pyserini.analysis import Analyzer, get_lucene_analyzer
# analyzer = get_lucene_analyzer(stemmer='porter', stopwords=False)
# se.reader.get_term_counts("spanish",analyzer) #(df,cf)

In [ ]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics,k=5, m=2, output_file="hey.txt")

In [ ]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("Results/run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

In [10]:
stats = se.reader.stats()
print(f"average terms per doc: {stats['total_terms']/stats['documents']}")

average terms per doc: 330.5510520235593


In [ ]:
doc = se.searcher.doc("FT921-3160")
text = doc.raw()
text

In [7]:
map = get_map_by_paths("Data/qrels_50_Queries", "Results/run.txt")

In [3]:
opti = Optimize()
# fb_terms_values = [5, 6, 8, 10, 15, 20]
# fb_docs_values = [5, 7, 10, 15]
# og_query_weight_values = [0.3, 0.4, 0.5, 0.6, 0.7]
mus = [200,300,400,500,600,700,800,900,1000,1100,1200]

opti.optimize_qld(topics, [20], [5], [0.6], mus, k=1000)

fb_terms=20, fb_docs=5, w=0.6, mu=200 -> MAP=0.259288
fb_terms=20, fb_docs=5, w=0.6, mu=300 -> MAP=0.264891
fb_terms=20, fb_docs=5, w=0.6, mu=400 -> MAP=0.264188
fb_terms=20, fb_docs=5, w=0.6, mu=500 -> MAP=0.263658
fb_terms=20, fb_docs=5, w=0.6, mu=600 -> MAP=0.262625
fb_terms=20, fb_docs=5, w=0.6, mu=700 -> MAP=0.259600
fb_terms=20, fb_docs=5, w=0.6, mu=800 -> MAP=0.256631
fb_terms=20, fb_docs=5, w=0.6, mu=900 -> MAP=0.254628
fb_terms=20, fb_docs=5, w=0.6, mu=1000 -> MAP=0.253824
fb_terms=20, fb_docs=5, w=0.6, mu=1100 -> MAP=0.251106
fb_terms=20, fb_docs=5, w=0.6, mu=1200 -> MAP=0.249275

BEST:
fb_terms=20, fb_docs=5, w=0.6, mu=300 -> MAP=0.264891
